# ViT编码器详解

## 📋 文档说明

本文档是ViT（Vision Transformer）编码器的详细理论讲解，比父目录的《视觉编码器详解》更加深入和详细。本文档将深入讲解ViT编码器的原理、数学推导和实现细节。通过本文档，你将能够：

1. **深入理解ViT编码器的原理**：从图像分块、线性投影到自注意力和Transformer编码器的全流程
2. **掌握自注意力的数学原理**：理解自注意力、多头注意力的数学定义、计算方法和几何意义
3. **理解ViT的架构设计**：理解Patch Embedding、位置编码、CLS Token、LayerNorm、残差连接等关键组件
4. **掌握ViT在VLA中的应用**：理解ViT编码器在VLA模型中的具体应用和优势
5. **了解ViT的变体和改进**：例如Swin Transformer、DeiT、ViT-L/14等

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过可视化图表和数学推导更好地理解ViT编码器的原理和过程。

**文档结构**：
- 父目录：视觉编码器详解（见../视觉编码器详解.ipynb）
- 本文档：ViT编码器详解（本文档）

---

## 📚 术语表（按出现顺序）

### 1. ViT编码器 (Vision Transformer Encoder)
- **中文名称**：ViT编码器
- **英文全称**：Vision Transformer Encoder
- **定义**：ViT编码器是基于Vision Transformer架构的视觉编码器，将图像分割成固定大小的图像块（patch），将每个图像块视为一个token，然后使用Transformer的自注意力机制处理这些token，提取图像特征。ViT编码器的优势在于：1）全局感受野：自注意力机制能够捕获图像中任意两个位置之间的关系，具有全局感受野；2）长距离依赖：能够捕获图像中的长距离依赖关系，不受卷积核大小的限制；3）并行计算：Transformer的并行计算能力，提高训练和推理效率；4）可扩展性：通过增加模型大小和数据量，能够持续提升性能。ViT编码器是现代VLA的重要方法，在需要全局感受野和长距离依赖的VLA任务中取得了很好的效果。
- **核心组成**：ViT编码器的核心组成包括：1）图像分块：将图像分割成固定大小的图像块，通常为16x16或32x32像素；2）线性投影：将每个图像块投影为固定维度的向量，作为token；3）位置编码：为每个token添加位置编码，保留图像的空间信息；4）Transformer编码器：使用多层Transformer编码器处理token，提取特征；5）自注意力机制：使用自注意力机制捕获token之间的关系；6）CLS Token：在token序列前添加CLS Token，用于全局特征汇聚；7）LayerNorm与残差连接：保证训练稳定性和梯度传播。ViT编码器通常使用预训练的权重初始化，然后在VLA任务上进行微调。
- **在VLA中的应用**：在VLA中，ViT编码器是视觉编码器的重要方法。VLA模型使用ViT编码器从输入图像中提取视觉特征，这些特征将被用于理解视觉场景、识别物体、理解关系等。ViT编码器的优势在于能够捕获图像中的长距离依赖关系，这对于理解复杂的视觉场景非常重要。在VLA训练过程中，ViT编码器通常使用预训练的权重初始化，然后在VLA任务上进行微调，以提高特征提取的质量和效率。ViT编码器的输出特征将与语言编码器的输出特征进行融合，生成多模态表示，最终用于动作生成。ViT编码器与ResNet编码器可以互补，某些VLA模型同时使用ResNet和ViT提取特征，融合两种方法的优势。
- **相关概念**：Transformer、自注意力机制、位置编码、视觉编码器、图像分块、CLS Token、LayerNorm、残差连接
- **首次出现位置**：本文档标题
- **深入学习**：参考[02_ViT编码器](./)和[Vision_Transformer特征提取详解](../../01_图像特征提取/03_Vision_Transformer/理论笔记/Vision_Transformer特征提取详解.ipynb)
- **直观理解**：想象ViT编码器就像将图像分成很多小块，然后让这些小块"互相交流"，通过"交流"理解图像的内容。ViT将图像分割成固定大小的图像块，每个图像块就像图像中的一个"单词"，然后使用Transformer的自注意力机制让这些"单词"互相"交流"，理解它们之间的关系。通过这种"交流"，ViT能够从图像中提取出丰富的特征表示，用于理解图像的内容。在VLA中，这些特征帮助模型理解视觉场景，从而生成相应的动作。

### 2. 图像分块 (Image Patching)
- **中文名称**：图像分块
- **英文全称**：Image Patching
- **定义**：图像分块是指将输入图像分割成固定大小的图像块（patch），例如16x16或32x32，然后将每个图像块展平并投影为固定维度的向量，作为Transformer的输入token。图像分块的作用是将二维图像转换为一维的token序列，使得Transformer可以处理图像数据。图像分块的优势在于：1）简单直接：不需要复杂的卷积操作；2）灵活性：可以选择不同的patch大小；3）与Transformer兼容：将图像转换为序列，直接使用NLP的Transformer架构。缺点是：1）丢失局部平移不变性：与卷积不同，分块后不具备平移不变性；2）patch大小过大可能丢失细节，过小则token过多，计算量大。
- **核心组成**：1）patch大小P：决定每个token包含的像素；2）分块操作：将图像分割为 \(\frac{H}{P} \times \frac{W}{P}\) 个patch；3）展平与线性投影：每个patch展平后经线性层映射到维度D；4）可选的CLS Token和位置编码添加。
- **在VLA中的应用**：在ViT编码器中，图像分块是输入转换的第一步。分块后的token序列输入Transformer编码器，提取全局特征。VLA模型依赖高质量的视觉token表示，分块大小和投影维度会影响特征质量和计算成本。
- **相关概念**：Patch Embedding、线性投影、位置编码、CLS Token、自注意力
- **首次出现位置**：本文档第1节
- **直观理解**：把一张图像切成许多小方块，每块当作一个“单词”，然后交给Transformer去理解这些“单词”之间的关系。

### 3. 位置编码 (Positional Encoding)
- **中文名称**：位置编码
- **英文全称**：Positional Encoding
- **定义**：位置编码为每个token添加位置信息，弥补Transformer对序列位置不敏感的问题。常见有正弦余弦位置编码（SPE）和可学习的位置编码（Learnable PE）。在ViT中，通常使用可学习的2D位置编码，将行列位置信息编码后与patch token相加。位置编码的作用是保持图像的空间结构，使模型知道“谁在左谁在右、谁在上谁在下”。
- **核心组成**：1）可学习位置向量矩阵（大小与token序列长度匹配）；2）行列索引映射到位置向量；3）与Patch Embedding相加或拼接；4）可选的相对位置偏置（Swin等）。
- **在VLA中的应用**：为视觉token提供空间位置信息，便于后续多模态对齐和动作决策。
- **相关概念**：正弦位置编码、相对位置编码、可学习位置编码、CLS Token
- **首次出现位置**：本文档第1节
- **直观理解**：给每个“单词”贴上它在图片里的坐标标签，让模型知道这些“单词”来自哪里。

### 4. 自注意力机制 (Self-Attention Mechanism)
- **中文名称**：自注意力机制
- **英文全称**：Self-Attention Mechanism
- **定义**：自注意力机制计算序列中任意两个位置的相关性，使模型能够关注全局关系。数学表示：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

其中Q、K、V分别由输入通过线性变换得到。自注意力的优势：1）全局感受野；2）长距离依赖；3）并行计算；4）可解释性（注意力权重）。
- **核心组成**：1）线性变换得到Q/K/V；2）注意力分数与softmax；3）加权求和得到输出；4）缩放因子\(\sqrt{d_k}\)。
- **在VLA中的应用**：用于ViT编码器，捕获图像中任意区域间的关系；在多模态对齐时也用于视觉-语言交互。
- **相关概念**：多头注意力、缩放点积注意力、Q/K/V、长距离依赖、全局感受野
- **首次出现位置**：本文档第2节
- **直观理解**：让每个“单词”查看其他所有“单词”，计算“我应该关注谁”，然后汇总信息。

### 5. 多头注意力 (Multi-Head Attention)
- **中文名称**：多头注意力
- **英文全称**：Multi-Head Attention
- **定义**：多头注意力将Q/K/V分成多个子空间，分别计算注意力，然后拼接，以多视角捕获关系。数学表示：

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O$$

其中 \(\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)\)。
- **核心组成**：1）多组投影矩阵 \(W_i^Q, W_i^K, W_i^V\)；2）并行计算多个注意力头；3）拼接后线性变换 \(W^O\)。
- **在VLA中的应用**：增强视觉token的关系建模，捕获不同尺度、不同语义的关联。
- **相关概念**：自注意力、Transformer编码器、头数h、维度分配
- **首次出现位置**：本文档第2节
- **直观理解**：多副“放大镜”从不同角度同时看同一张图，最后把信息合并。

### 6. CLS Token (Class Token)
- **中文名称**：CLS Token
- **英文全称**：Classification Token
- **定义**：CLS Token是一个可学习的全局特征token，预置在token序列的开头，汇聚整张图像的信息，常用于分类或下游全局任务。在ViT中，最终使用CLS Token的输出作为全局特征。
- **核心组成**：1）可学习向量；2）与patch token一起输入Transformer；3）在每层通过自注意力与其他token交互。
- **在VLA中的应用**：作为全局视觉特征，与语言特征融合或直接用于动作决策。
- **相关概念**：全局池化、token序列、自注意力
- **首次出现位置**：本文档第1节
- **直观理解**：在“单词”前面加一个“总结者”，最后用它代表整张图片的摘要。



In [ ]:
# ============================================
# 导入必要的库
# ============================================
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA

plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-darkgrid')
print("环境准备完成！")


## 📋 概述

### 什么是ViT编码器

ViT编码器是基于Transformer的视觉编码器，将图像切分为patch token，再用自注意力建模全局关系。

### 为什么重要

1. **全局感受野**：自注意力直接建模长距离关系
2. **长距离依赖**：捕获复杂场景的全局交互
3. **并行计算**：Transformer的高吞吐特性
4. **可扩展性**：增大模型和数据即可提升性能

### 学习目标

1. 理解图像分块与Patch Embedding
2. 掌握自注意力与多头注意力的数学原理
3. 理解位置编码、CLS Token、LayerNorm、残差连接
4. 理解ViT在VLA中的应用与变体

---

## 1. 图像分块与Patch Embedding

### 1.1 什么是图像分块

**直观理解**：把一张图像切成很多小方块，每块当作一个“单词”，交给Transformer去理解它们的关系。

### 1.2 为什么需要图像分块

1. 将2D图像转为1D token序列，便于Transformer处理
2. 可选择patch大小，平衡细节与计算量
3. 与NLP的Transformer架构直接兼容

### 1.3 数学表示

假设图像大小为 \(H \times W\)，patch大小为 \(P \times P\)：

- patch 数量：\(N = \frac{H}{P} \times \frac{W}{P}\)
- 每个patch展平后长度：\(P^2 \cdot C\)（C为通道数）
- 线性投影到维度D： \(z_i = \text{Linear}(\text{Flatten}(\text{patch}_i)) \in \mathbb{R}^D\)

### 1.4 具体计算示例（数值版）

以 \(H = W = 32, P = 16, C = 3, D = 768\)：

- patch 数量：\(N = (32/16)^2 = 4\) 个
- 每个patch展平长度：\(16 \times 16 \times 3 = 768\)
- 线性投影：每个patch乘以 \(768 \times 768\) 的矩阵（可共享权重）得到 token 向量

### 1.5 可视化：图像分块与Patch Embedding


In [ ]:
# ============================================
# 图像分块与Patch Embedding可视化
# ============================================

# 构造一张简单的示例图像（32x32，RGB）
H = W = 32
P = 16
C = 3
img = np.zeros((H, W, C))
img[0:16, 0:16, :] = [1.0, 0.7, 0.7]   # 左上红
img[0:16, 16:32, :] = [0.7, 1.0, 0.7]   # 右上绿
img[16:32, 0:16, :] = [0.7, 0.7, 1.0]   # 左下蓝
img[16:32, 16:32, :] = [1.0, 1.0, 0.7]   # 右下黄

# 分块
def split_patches(image, P):
    H, W, C = image.shape
    patches = []
    for i in range(0, H, P):
        for j in range(0, W, P):
            patches.append(image[i:i+P, j:j+P, :])
    return np.stack(patches, axis=0)

patches = split_patches(img, P)  # (N, P, P, C)
N = patches.shape[0]

# 展平 + 线性投影（用随机权重演示）
D = 8  # 为展示简化维度
flattened = patches.reshape(N, -1)  # (N, P*P*C)
W_proj = np.random.randn(flattened.shape[1], D) * 0.1
embedded = flattened @ W_proj  # (N, D)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 原图
axes[0].imshow(img)
axes[0].set_title('原始图像 32x32', fontsize=12, fontweight='bold')
axes[0].axis('off')

# 分块示意
axes[1].imshow(img)
for i in range(0, H, P):
    axes[1].axhline(i-0.5, color='white', linewidth=2)
for j in range(0, W, P):
    axes[1].axvline(j-0.5, color='white', linewidth=2)
axes[1].set_title(f'分块示意 (P={P})，共{N}个patch', fontsize=12, fontweight='bold')
axes[1].axis('off')

# Token可视化（用PCA降到2D显示）
pca = PCA(n_components=2)
embedded_2d = pca.fit_transform(embedded)
axes[2].scatter(embedded_2d[:, 0], embedded_2d[:, 1], c=['r','g','b','y'], s=120)
for i in range(N):
    axes[2].text(embedded_2d[i,0], embedded_2d[i,1], f'Patch{i}', ha='center', va='center', fontsize=10)
axes[2].set_title('Patch Embedding (PCA到2D)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('PC1')
axes[2].set_ylabel('PC2')

plt.tight_layout()
plt.show()

print("=" * 60)
print("图像分块与Patch Embedding说明：")
print("=" * 60)
print(f"1. 原图被分成 {N} 个 patch，每个大小 {P}x{P}，共 {P*P*C} 维")
print("2. 每个 patch 展平后经线性层映射到维度 D（示例中 D=8）")
print("3. 不同颜色的块映射到不同的 token 向量")
print("=" * 60)


In [ ]:
# ============================================
# 位置编码可视化（示例：可学习位置编码）
# ============================================

N = 4  # token数量（与前面的示例一致）
D = 8
# 随机初始化位置编码
pos_embed = np.random.randn(N, D) * 0.1

pca = PCA(n_components=2)
pos_2d = pca.fit_transform(pos_embed)

fig, ax = plt.subplots(figsize=(6, 6))
colors = ['r', 'g', 'b', 'y']
for i in range(N):
    ax.scatter(pos_2d[i,0], pos_2d[i,1], c=colors[i], s=120)
    ax.text(pos_2d[i,0], pos_2d[i,1], f'Pos{i}', ha='center', va='center', fontsize=10)
ax.set_title('可学习位置编码 (PCA到2D)', fontsize=12, fontweight='bold')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
plt.show()

print("=" * 60)
print("位置编码说明：")
print("=" * 60)
print("1. 每个token有一个位置向量，与patch embedding相加")
print("2. 可学习位置编码能适应任务；也可用正弦/相对位置编码")
print("3. 位置编码保证空间结构信息不丢失")
print("=" * 60)


## 3. 自注意力与多头注意力

### 3.1 自注意力的数学推导

给定输入token矩阵 \(X \in \mathbb{R}^{N \times D}\)：
1. 线性变换：\(Q = XW^Q, K = XW^K, V = XW^V\)
2. 注意力分数：\(A = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})\)
3. 输出：\(Y = AV\)

### 3.2 缩放因子 \(\sqrt{d_k}\) 的作用

防止点积随维度增大而过大导致softmax梯度饱和。

### 3.3 多头注意力

将 \(D\) 维分成 \(h\) 个子空间，每个子空间做注意力，再拼接。

### 3.4 具体计算示例（数值版）

设 N=4, D=8, h=2：
- 每头维度 \(d_h = D/h = 4\)
- 计算每个头的注意力分数与输出，再拼接。


In [ ]:
# ============================================
# 自注意力数值示例 (N=4, D=8, h=2)
# ============================================
np.random.seed(0)

N, D, h = 4, 8, 2
d_h = D // h
X = np.random.randn(N, D)  # 输入token

# 随机权重
WQ = np.random.randn(D, D)
WK = np.random.randn(D, D)
WV = np.random.randn(D, D)

Q = X @ WQ  # (N, D)
K = X @ WK
V = X @ WV

# 分头
Qs = Q.reshape(N, h, d_h).transpose(1,0,2)  # (h, N, d_h)
Ks = K.reshape(N, h, d_h).transpose(1,0,2)
Vs = V.reshape(N, h, d_h).transpose(1,0,2)

heads_out = []
for head in range(h):
    Qi = Qs[head]           # (N, d_h)
    Ki = Ks[head]           # (N, d_h)
    Vi = Vs[head]           # (N, d_h)
    scores = Qi @ Ki.T / np.sqrt(d_h)  # (N, N)
    attn = np.exp(scores) / np.exp(scores).sum(axis=-1, keepdims=True)
    out = attn @ Vi         # (N, d_h)
    heads_out.append(out)

# 拼接
Y = np.concatenate(heads_out, axis=-1)  # (N, D)

print("输入X形状:", X.shape)
print("注意力输出Y形状:", Y.shape)
print("示例：第0个token的注意力权重（head0）:\n", attn[0])

plt.figure(figsize=(6,4))
plt.imshow(attn, cmap='viridis')
plt.colorbar()
plt.title('注意力权重 (head0, 4x4)')
plt.xlabel('Key index')
plt.ylabel('Query index')
plt.show()

print("=" * 60)
print("自注意力数值示例说明：")
print("=" * 60)
print("1. 展示了N=4, D=8, h=2的多头注意力计算流程")
print("2. 注意力权重矩阵可视化，显示token间的相关性")
print("3. 缩放因子 sqrt(d_h) 保持softmax稳定")
print("=" * 60)


## 4. Transformer编码器层

### 4.1 组成
- 多头注意力（MHA）
- 残差连接 + LayerNorm
- 前馈网络（FFN，两层MLP，中间激活GELU）
- 残差连接 + LayerNorm

### 4.2 数学表示
设输入为 \(X\)：
1. \(X' = \text{LayerNorm}(X + \text{MHA}(X))\)
2. \(Y = \text{LayerNorm}(X' + \text{FFN}(X'))\)

### 4.3 FFN计算
\(\text{FFN}(x) = W_2 \sigma(W_1 x + b_1) + b_2\)，通常 \(d_{ff} = 4d\)。

### 4.4 可视化：ViT Block结构


In [ ]:
# ============================================
# ViT Block结构可视化
# ============================================
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('ViT Block (Transformer Encoder 层)', fontsize=14, fontweight='bold', pad=15)

# 输入
input_box = FancyBboxPatch((4, 11), 2, 0.8, boxstyle="round,pad=0.1", 
                           facecolor='lightblue', edgecolor='black', linewidth=2)
ax.add_patch(input_box)
ax.text(5, 11.4, '输入 X', ha='center', va='center', fontsize=11, fontweight='bold')

# MHA
mha_box = FancyBboxPatch((4, 9.5), 2, 1, boxstyle="round,pad=0.1", 
                         facecolor='lightgreen', edgecolor='black', linewidth=2)
ax.add_patch(mha_box)
ax.text(5, 10, '多头注意力 MHA', ha='center', va='center', fontsize=10)

# 残差 + LN1
ln1_box = FancyBboxPatch((4, 8), 2, 0.8, boxstyle="round,pad=0.1", 
                         facecolor='lightyellow', edgecolor='black', linewidth=2)
ax.add_patch(ln1_box)
ax.text(5, 8.4, '残差 + LayerNorm', ha='center', va='center', fontsize=10)

# FFN
ffn_box = FancyBboxPatch((4, 6.5), 2, 1, boxstyle="round,pad=0.1", 
                         facecolor='lightcoral', edgecolor='black', linewidth=2)
ax.add_patch(ffn_box)
ax.text(5, 7, '前馈网络 FFN', ha='center', va='center', fontsize=10)

# 残差 + LN2
ln2_box = FancyBboxPatch((4, 5), 2, 0.8, boxstyle="round,pad=0.1", 
                         facecolor='lightyellow', edgecolor='black', linewidth=2)
ax.add_patch(ln2_box)
ax.text(5, 5.4, '残差 + LayerNorm', ha='center', va='center', fontsize=10)

# 输出
output_box = FancyBboxPatch((4, 3.5), 2, 0.8, boxstyle="round,pad=0.1", 
                            facecolor='lightblue', edgecolor='black', linewidth=2)
ax.add_patch(output_box)
ax.text(5, 3.9, '输出 Y', ha='center', va='center', fontsize=11, fontweight='bold')

# 箭头
steps = [(5, 11), (5, 10.5), (5, 9.5), (5, 8.5), (5, 7.5), (5, 6.5), (5, 5.5), (5, 5), (5, 4.3)]
for i in range(len(steps)-1):
    arrow = FancyArrowPatch(steps[i], steps[i+1], arrowstyle='->', 
                           mutation_scale=15, linewidth=1.5, color='black')
    ax.add_patch(arrow)

plt.tight_layout()
plt.show()

print("=" * 60)
print("ViT Block结构说明：")
print("=" * 60)
print("1. 多头注意力后做残差与LayerNorm")
print("2. FFN后再次残差与LayerNorm")
print("3. 两次残差连接保证梯度稳定")
print("4. FFN通常维度放大4倍提升表达能力")
print("=" * 60)


## 5. CLS Token与输出

### 5.1 CLS Token的作用
- 作为全局汇聚token，与所有patch交互
- 最终用于分类或下游全局表征

### 5.2 对比：CLS池化 vs GAP
- CLS：通过注意力聚合全局信息，可学习
- GAP：对特征图做均值池化，简单但固定

### 5.3 流程小结
1. [CLS] + patch tokens
2. 位置编码相加
3. 多层Transformer编码器
4. 取CLS输出作为全局特征（或平均所有token）



## 6. ViT在VLA中的应用

### 6.1 使用流程
1. 图像分块 + Patch Embedding
2. 位置编码 + 可选CLS Token
3. 多层Transformer编码器
4. 取CLS或平均池化得到视觉特征
5. 与语言特征融合，输入多模态模块

### 6.2 优势
- 捕获长距离依赖，适合复杂场景理解
- 并行计算，高吞吐
- 预训练生态成熟（ViT-B/16, ViT-L/14, ViT-H/14等）

### 6.3 劣势与折中
- 计算量随patch数二次增长（注意力为O(N^2)）
- 对局部平移不变性不如CNN，需要更大数据或归纳偏置

### 6.4 典型变体
- **DeiT**：数据高效训练，蒸馏token
- **Swin**：层次化、窗口注意力，线性复杂度，保留局部性
- **ViT-L/14, ViT-H/14**：更大模型，适合高分辨率与下游微调
- **Hybrid**：CNN特征+Transformer（如R50+ViT-B/16）



## 7. 小结

1. ViT通过分块+自注意力实现全局建模，具备长距离依赖和全局感受野
2. 核心组件：Patch Embedding、位置编码、CLS Token、MHA、FFN、残差+LayerNorm
3. 计算复杂度O(N^2)，需注意patch大小与分辨率的折中
4. 在VLA中常与语言特征融合，或与CNN特征互补

---

**文档完成时间**：2025-01-27  
**文档版本**：v1.0  
**维护者**：AI助手

**相关文档**：
- 父目录：视觉编码器详解（见../视觉编码器详解.ipynb）
- Vision Transformer特征提取详解（见../../01_图像特征提取/03_Vision_Transformer/理论笔记/Vision_Transformer特征提取详解.ipynb）
- ResNet编码器详解（见../01_ResNet编码器/理论笔记/ResNet编码器详解.ipynb）
